In [14]:
# ============================================================
# CreatorCopilot: Final Robust Version (2026)
# Fixes: 404 Model Errors & 429 Quota/Rate Limit Errors
# ============================================================

# 1. Install/Update packages
!pip install --quiet -U google-generativeai pandas

import os
import time
import random
import pandas as pd
import google.generativeai as genai
from kaggle_secrets import UserSecretsClient

# ------------------ API & Model Setup ----------------------
try:
    user_secrets = UserSecretsClient()
    GEMINI_API_KEY = user_secrets.get_secret("GEMINI_API_KEY")
    genai.configure(api_key=GEMINI_API_KEY)
    print("DEBUG: API Key retrieved.")
except:
    print("DEBUG: API Key not found in Kaggle Secrets. Using placeholder (Mock data will be used).")
    GEMINI_API_KEY = "YOUR_GEMINI_API_KEY_HERE"

# Logic to pick the best available model (fixes 404)
try:
    available_models = [m.name for m in genai.list_models()]
    if 'models/gemini-2.5-flash' in available_models:
        MODEL_ID = 'models/gemini-2.5-flash'
    else:
        MODEL_ID = 'models/gemini-1.5-flash'
    print(f"DEBUG: Using model -> {MODEL_ID}")
except:
    MODEL_ID = 'models/gemini-1.5-flash'

# ------------------- Robust LLM Helper ---------------------
def call_llm(system_prompt: str, user_prompt: str, max_retries: int = 5) -> str:
    """Calls Gemini with exponential backoff for rate limiting (429)."""
    if GEMINI_API_KEY == "YOUR_GEMINI_API_KEY_HERE":
        return "Mock Response: API Key is missing."

    model = genai.GenerativeModel(
        model_name=MODEL_ID,
        system_instruction=system_prompt
    )

    for attempt in range(max_retries):
        try:
            response = model.generate_content(user_prompt)
            return (response.text or "").strip()
        except Exception as e:
            err_msg = str(e).lower()
            if "429" in err_msg or "quota" in err_msg or "limit" in err_msg:
                # Exponential backoff: 2, 4, 8, 16... seconds
                wait_time = (2 ** (attempt + 1)) + random.uniform(0, 1)
                print(f"  [Rate Limit] Attempt {attempt+1} failed. Retrying in {wait_time:.1f}s...")
                time.sleep(wait_time)
            else:
                return f"API Error: {str(e)}"
    
    return "Error: Max retries exceeded (Rate Limit)."

# ------------------- Agent Logic ---------------------------
def generate_ideas(niche, audience, goal, num_ideas=2):
    sys = "You are a YouTube strategist. List ideas as: 1. Title: ... Description: ... Hook: ..."
    user = f"Niche: {niche}, Audience: {audience}, Goal: {goal}, Count: {num_ideas}"
    raw = call_llm(sys, user)
    # Parsing into list
    lines = [l.strip() for l in raw.split('\n') if l.strip()]
    return [{"id": i+1, "raw": l} for i, l in enumerate(lines[:num_ideas])]

# ------------------- Orchestrator --------------------------
def run_creator_copilot(niche, audience, goal, num_ideas=2):
    print(f"\n--- Processing: {niche} ---")
    ideas = generate_ideas(niche, audience, goal, num_ideas)
    records = []
    
    for idea in ideas:
        print(f"  > Planning Idea {idea['id']}...")
        
        # Briefing
        brief = call_llm("You are a YT Planner. Create a brief with Title, Hook, and Outline.", idea['raw'])
        
        # Scripting
        script = call_llm("Write a high-energy 45s YouTube Short script.", brief)
        
        # Thumbnailing
        thumb = call_llm("Create short thumbnail text and a visual concept description.", brief)
        
        records.append({
            "niche": niche,
            "idea": idea["raw"],
            "brief": brief,
            "script": script,
            "thumbnail": thumb
        })
        
        # Proactive throttle to stay under Free Tier RPM
        time.sleep(2) 
        
    return pd.DataFrame(records)

# ------------------- Execution -----------------------------
df_mine = run_creator_copilot("Minecraft Survival", "Beginner Players", "Day 1 Survival", num_ideas=2)
df_py = run_creator_copilot("Python Tips", "Coding Students", "Confidence building", num_ideas=2)

df_final = pd.concat([df_mine, df_py], ignore_index=True)
df_final.to_csv("creatorcopilot_final.csv", index=False)

print("\n=== SUCCESS: DATA GENERATED ===")
display(df_final.head())

DEBUG: API Key retrieved.
DEBUG: Using model -> models/gemini-2.5-flash

--- Processing: Minecraft Survival ---
  > Planning Idea 1...
  > Planning Idea 2...

--- Processing: Python Tips ---
  > Planning Idea 1...
  > Planning Idea 2...

=== SUCCESS: DATA GENERATED ===


,niche,idea,brief,script,thumbnail
0,Minecraft Survival,Here are two YouTube video ideas for Minecraft...,Excellent! Here are the briefs for your two Yo...,"Okay, this is going to be a blast! For a 45-se...",Here are the thumbnail text and visual concept...
1,Minecraft Survival,---,Here's a brief for a YouTube video focused on ...,## UNLOCK YOUR POTENTIAL: 5 Habits to Boost Pr...,Here are the thumbnail text and visual concept...
2,Python Tips,"Here are two video ideas for your ""Python Tips...","Here are the YouTube briefs for your ""Python T...",## YouTube Short Script: Debugging with Print ...,Here are the thumbnail text and visual concept...
3,Python Tips,---,"Okay, I'll create a brief for a YouTube video ...","Okay, this is going to be **FAST**, **PUNCHY**...",Here are the short thumbnail text and visual c...
